<a href="https://colab.research.google.com/github/gzholtkevych/Design-Compilers-for-DSL/blob/main/RegEx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<H1><b>РЕГУЛЯРНІ ВИРАЗИ та СКІНЧЕННІ АКЦЕПТОРИ</b></H1>

# Сервісні функції та класи

In [1]:
from typing import Set
from typing_extensions import Self


def guard(condition: bool, failure: Exception) -> None:
    """computes 'condition' and if the result is False
    raises exception 'failure'
    """
    if not condition:
        raise failure


# Символи (токени)

Будемо вважати зафіксованим певний алфавіт $\mathrm A$, який містить сутності, що називаються *символами* (або *токенами*) і які моделюються як екземпляри наступного класу.

In [6]:
class Chr(str):

    def __init__(self, ch: str) -> Self:
        guard(isinstance(ch, str),
              TypeError("invalid argumnet of Char-constructor"))
        self._data = ch

    def __str__(self) -> str:
        return f"'{self._data}'"

    def __repr__(self) -> str:
        return f"Chr('{self._data}')"

    def __eq__(self, another: Self) -> bool:
        if isinstance(another, Chr):
            return self._data == another._data
        return False

# Регулярні вирази

Регулярні вирази визначаються для певного алфавіту $\mathrm A$, який будемо вважати зафіксованим як об'єкти, що будуються *виключно* за наступними правилами

1. ***Правила побудови константних регулярних виразів***

    1.1. $0$ є регулярним виразом,

    1.2. $1$ є регулярним виразом,

    1.3. $a$ для будь-якого $a\in\mathrm A$ є регулярним виразом.
2. ***Правило побудови замкнекння Кліні***. Якщо вже є побудованим регулярний вираз $e$, тоді $(e*)$ є регулярним виразом, який називають ***замкненням Кліні*** виразу $e$.
3. ***Правило побудови конкатенації***. Якщо вже є побудованими регулярні вирази $e_1$ та $e_2$, тоді $(e_1\ e_2)$ є регулярним виразом, який називають ***конкатенацією*** виразів $e_1$ та $e_2$.
4. ***Правило побудови альтернації***. Якщо вже є побудованими регулярні вирази $e_1$ та $e_2$, тоді $(e_1\mid e_2)$ є регулярним виразом, який називають ***альтернацією*** виразів $e_1$ та $e_2$.

## Базовий клас регулярного виразу

Базовий клас моделює регулярні вирази в цілому. Він відповідальний за:

- визначення виду регулярного виразу (властивість `kind`);
- представлення регулярного виразу у вигляді рядка (метод `__str__`);
- визначення множини символів, що входять у регулярний вираз (властивість `chars`);
- визначення чи належить порожній ланцюжок мові регулярного виразу (влістивість `containsEmptyChain`).

Остання властивість обчислюється за правилами

1. $0.\mathtt{containsEmptyChain}=\mathtt{False}$;
1. $1.\mathtt{containsEmptyChain}=\mathtt{True}$;
1. $a.\mathtt{containsEmptyChain}=\mathtt{False}$ для будь-якого $a\in\mathrm{A}$;
1. $(e*).\mathtt{containsEmptyChain}=\mathtt{True}$ для будь-якого регулярного виразу $e$;
1. $(e_1\ e_2).\mathtt{containsEmptyChain}=e_1.\mathtt{containsEmptyChain}\land e_2.\mathtt{containsEmptyChain}$ для будь-яких регулярних виразів $e_1$ та $e_2$;
1. $(e_1\mid e_2).\mathtt{containsEmptyChain}=e_1.\mathtt{containsEmptyChain}\lor e_2.\mathtt{containsEmptyChain}$ для будь-яких регулярних виразів $e_1$ та $e_2$.

In [7]:
class ReX:

    @property
    def kind(self) -> str:
        """returns the kind of the regulat expression:
        'empty' for the empty expression
        'nil'   for the nil expression
        'char'  for a char expression
        'ast'   for an expression created as Kleene closure
        'cat'   for an expression created as concatenation
        'alt'   for an expression created as alternation
        """
        if isinstance(self, Empty):
            return 'empty'
        if isinstance(self, Nil):
            return 'nil'
        if isinstance(self, Char):
            return 'char'
        if isinstance(self, Ast):
            return 'ast'
        if isinstance(self, Cat):
            return 'cat'
        if isinstance(self, Alt):
            return 'alt'

    def __str__(self) -> str:
        """returns the str-representation of a ReX-instance
        """
        if self.kind == 'empty':
            return '0'
        if self.kind == 'nil':
            return '1'
        if self.kind == 'char':
            return f"'{self._data}'"
        if self.kind == 'ast':
            return f'({self._data})*'
        if self.kind == 'cat':
            return f'({self._data[0]} {self._data[1]})'
        if self.kind == 'alt':
            return f'({self._data[0]} | {self._data[1]})'

    @property
    def chars(self) -> Set[Chr]:
        if self.kind == 'empty' or self.kind == 'nil':
            return set()
        if self.kind == 'char':
            return {self._data,}
        if self.kind == 'ast':
            return self._data.chars
        if self.kind == 'cat' or self._data == 'alt':
            temp = self._data[0].chars
            temp |= self._data[1].chars
            return temp

    @property
    def containsEmptyChain(self) -> bool:
        if self.kind == 'empty':
            return False
        if self.kind == 'nil':
            return True
        if self.kind == 'char':
            return False
        if self.kind == 'ast':
            return True
        if self.kind == 'cat':
            flag = self._data[0].containsEmptyChain
            flag = flag and self._data[1].containsEmptyChain
            return flag
        if self.kind == 'alt':
            flag = self._data[0].containsEmptyChain
            flag = flag or self._data[1].containsEmptyChain
            return flag

## Підкласи примітивних регулярних виразів

Клас, екземплярами якого є вирази, що дорівнюють $0$.

In [8]:
class Empty(ReX):
    pass

Клас, екземплярами якого є вирази, що дорівнюють $1$.

In [12]:
class Nil(ReX):
    pass

Клас, екземплярами якого є вирази, що мають вид $a$ для $a\in\mathrm A$.

In [13]:
class Char(ReX):

    def __init__(self, ch: Chr) -> Self:
        guard(isinstance(ch, Chr),
              TypeError("the argument type of Char-constructor is invalid"))
        self._data = ch

## Підклас замкнення Кліні

Клас, екземплярами якого є вирази, що мають вид $(e*)$ для вже побудованого регулярного виразу $e$.

In [14]:
class Ast(ReX):

    def __init__(self, e: ReX) -> Self:
        guard(isinstance(e, ReX),
              TypeError("the argument of Ast-constructor is invalid"))
        self._data = e

## Підклас конкатенації

Клас, екземплярами якого є вирази, що мають вид $(e_1\ e_2)$ для вже побудованих регулярних виразів $e1$ та $e_2$.

In [15]:
class Cat(ReX):

    def __init__(self, e1: ReX, e2: ReX) -> Self:
        guard(isinstance(e1, ReX),
              TypeError("the first argument of Cat-constructor is invalid"))
        guard(isinstance(e2, ReX),
              TypeError("the second argument of Cat-constructor is invalid"))
        self._data = (e1, e2)

## Підклас альтернації

Клас, екземплярами якого є вирази, що мають вид $(e_1\mid e_2)$ для вже побудованих регулярних виразів $e_1$ та $e_2$.

In [16]:
class Alt(ReX):

    def __init__(self, e1: ReX, e2: ReX) -> Self:
        guard(isinstance(e1, ReX),
              TypeError("the first argument of Alt-constructor is invalid"))
        guard(isinstance(e2, ReX),
              TypeError("the second argument of Alt-constructor is invalid"))
        self._data = (e1, e2)

## Оператори над регулярними виразами

З кожним регулярним виразом $e$ пов'язується мова $[\hspace{-1.5pt}[e]\hspace{-1.5pt}]$ над алфавітом $\mathrm A$, яка будується наступним чином

1. $[\hspace{-1.5pt}[0]\hspace{-1.5pt}]=\emptyset$;
1. $[\hspace{-1.5pt}[1]\hspace{-1.5pt}]=\{[\,]\}$;
1. $[\hspace{-1.5pt}[a]\hspace{-1.5pt}]=\{[a]\}$ для $a\in\mathrm A$;
1. $[\hspace{-1.5pt}[(e*)]\hspace{-1.5pt}]=[\hspace{-1.5pt}[e]\hspace{-1.5pt}]^\ast$ для регулярного виразу $e$;
1. $[\hspace{-1.5pt}[(e_1\ e_2)]\hspace{-1.5pt}]=[\hspace{-1.5pt}[e_1]\hspace{-1.5pt}]\cdot [\hspace{-1.5pt}[e_2]\hspace{-1.5pt}]$ для регулярних виразів $e_1$ та $e_2$;
1. $[\hspace{-1.5pt}[(e_1\mid e_2)]\hspace{-1.5pt}]=[\hspace{-1.5pt}[e_1]\hspace{-1.5pt}]\cup [\hspace{-1.5pt}[e_2]\hspace{-1.5pt}]$ для регулярних виразів $e_1$ та $e_2$.

Будемо називати два регулярні вирази $e_1$ та $e_2$ *семантично рівними* (позначаємо через $e_1\simeq e_2$), якщо $[\hspace{-1.5pt}[e_1]\hspace{-1.5pt}]=[\hspace{-1.5pt}[e_2]\hspace{-1.5pt}]$.

### Оператор спрощення

Оператор спрощення будує для регулярного виразу $e$ регулярний вираз $e'$ такий, що $e'\simeq e$, використовуючи наступні тотожності

1. $(0*)=1$;
1. $(1*)=1$;
1. $((e*)*)=(e*)$;
1. $(0\ e)=0=(e\ 0)$;
1. $(1\ e)=e=(e\ 1)$;
1. $(0\mid e)=e=(e\mid 0)$.

Тут всюди $e$ є регулярним виразом.

In [18]:
def simplify(e: ReX) -> ReX:
    """returns the result of simplifying 'e'"""
    guard(isinstance(e, ReX),
          TypeError("the argument of 'simplify' is invalid"))
    if e.kind == 'ast':
        if e._data.kind == 'empty' or e._data == 'nil':
            return Nil()
        if e._data.kind == 'ast':
            return simplify(e._data)
        return Ast(simplify(e._data))
    if e.kind == 'cat':
        if e._data[0].kind == 'empty' or e._data[1].kind == 'empty':
            return Empty()
        if e._data[0].kind == 'nil':
            return simplify(e._data[1])
        if e._data[1].kind == 'nil':
            return simplify(e._data[0])
        return Cat(simplify(e._data[0]), simplify(e._data[1]))
    if e.kind == 'alt':
        if e._data[0].kind == 'empty':
            return simplify(e._data[1])
        if e._data[1].kind == 'empty':
            return simplify(e._data[0])
        return Alt(simplify(e._data[0]), simplify(e._data[1]))
    return e

### Оператор диференціювання за Бжозовськи

Оператор диференціювання за Бжозовськи регулярного виразу $e$ за символом $a\in\mathrm{A}$ будує похідну Бжозовськи $a^{-1}\cdot e$, яка обчислюється за правилами

1. $a^{-1}\cdot 0=0$;
1. $a^{-1}\cdot 1=0$;
1. $a^{-1}\cdot b=1\texttt{ if $b=a$ else }0$;
1. $a^{-1}\cdot(e*)=((a^{-1}\cdot e)\ (e*))$;
1. $a^{-1}\cdot(e_1\ e_2)=(((a^{-1}\cdot e_1)\ e_2)\mid a^{-1}\cdot e_2)\texttt{ if $e_1.\mathtt{containsEmptyChain}$ else }((a^{-1}\cdot e_1)\ e_2)$;
1. $a^{-1}\cdot(e_1\mid e_2)=(a^{-1}\cdot e_1\mid a^{-1}\cdot e_2)$.



In [20]:
def differentiate(e: ReX, ch: Chr):
    """computes Bfzozowski's derevative of 'e' by 'ch'"""
    guard(isinstance(e, ReX),
          TypeError("the type of the first argument "
                    "of 'differentiate' is invalid"))
    guard(isinstance(ch, Chr),
          TypeError("the type of the second argument "
                    "of 'differentiate' is invalid"))
    if ch not in e.chars:
        return Empty()
    e = simplify(e)
    if e.kind == 'char':
        if e._data == ch:
            return Nil()
        return Empty()
    if e.kind == 'ast':
        temp = differentiate(e._data, ch)
        temp = Cat(temp, e)
        return simplify(temp)
    if e.kind == 'cat':
        temp = differentiate(e._data[0], ch)
        temp = Cat(temp, e._data[1])
        if e._data[0].containsEmptyChain:
            tempp = differentiate(e._data[1], ch)
            temp = Alt(temp, tempp)
        return simplify(temp)
    if e.kind == 'alt':
        temp = differentiate(e._data[0], ch)
        tempp = differentiate(e._data[1], ch)
        temp = Alt(temp, tempp)
        return simplify(temp)

# Скінченні акцептори

In [ ]:
class Acceptor:

    def __init__(self, table: list[tuple[dict[Chr, int], bool]]) -> Self:
        """creates the acceptor with 'table', which is a list of a tuple.
        The 'table' indeces corresponds to acceptor states.
        Each 'table' item contains two component: transition dictionary and
        accepting flag.
        Keys of a transition dictionary are input chars and values are do(i, c).
        The accepting flag is true for accepting state and false for otherwise.
        """
        guard(isinstance(table, list),
              TypeError("invalid type of 'table'"))
        guard(all(isinstance(item, tuple) and len(item) == 2 for item in table),
              TypeError("invalid table item type"))
        guard(all(isinstance(item[1], bool) for item in table),
              TypeError("invalid type of accepting flag in 'table'"))
        guard(all(isinstance(item[0], dict) for item in table),
              TypeError("invalid type of a transition dictionary in 'table'"))
        self._data = table

    def run(self, state: int, control: list[Chr]) -> None | int:
        """simulates behavior of the acceptor beginning at 'state' under
        'control'
        """
        guard(isinstance(state, int) and 0 <= state < len(self._data),
              ValueError("invalid initial state of 'run'"))
        guard(isinstance(control, list) and
              all(isinstance(char, Chr) for char in control),
              TypeError("invalid control of 'run'"))
        if not control:
            return state
        char, control = control[0], control[1:]
        try:
            state = self._data[state][char]
            return self.run(state, control)
        except:
            return None


In [ ]:
a, b = Chr('a'), Chr('b')
flip_flop = Acceptor([
    ({a: 0, b: 1}, True),
    ({a: 1, b: 0}, False)
])

print(flip_flop._data)
print(flip_flop.run(0, [a, a, b]))

[({Chr('a'): 0, Chr('b'): 1}, True), ({Chr('a'): 1, Chr('b'): 0}, False)]
None
